In [71]:
# First, we upload the data: commom AI questions as an example for am educational content 
# I used the Json format for the data file as it is easy to write/read 
# Then we extract the training data (The questions), taregt labels (Classifying the data into categories: greetings,
#thanking ... ), labels and answers


import json 

with open('Hypatiama.json') as file:
    data = json.load(file)
    
training_data = []
target_labels = []
labels = []
answers = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_data.append(pattern)
        target_labels.append(intent['tag'])
    answers.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
number_classes = len(labels)



In [72]:
# We use the LabelEncoder from sklearn to encode the target labels to values between 0 and number_classes-1 (#sklearn)


from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(target_labels)
target_labels = label_encoder.transform(target_labels)
print(target_labels)

[14 14 14 14 14 13 13 13 17 17 17 17 12 12 12 12 16 16 16 15 15 15 15 15
 15 10 10 10 10 11 11 11 11  9  9  9  9  7  7  7  7  0  0  0  0  4  4  4
  4  8  8  8  8  1  1  1  1  2  2  2  5  5  5  3  3  3  6  6  6]


In [73]:
# We use the NLP methods Tokenization( breaking the data into chunks)
# We define a limit for the vacabulary size( Thanks to tokenization) 
# To handle the out of Vocabulary we use OOV 
#Pads sequences to the same length. This function transforms a list (of length num_samples )
#of sequences (lists of integers) into a 2D Numpy array of shape (num_samples, num_timesteps) (## Keras)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
oov_token = "<OOV>"
vocabulary_size = 10000
embedding_dim = 16
max_length = 30


tokenizer = Tokenizer(num_words=vocabulary_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_data)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_data)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_length)

In [74]:
# Training : we use a neural network, we import the model Sequential from the Keras then we define the architecture 


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_dim, input_length=max_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(number_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 30, 16)            160000    
                                                                 
 global_average_pooling1d_9   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_27 (Dense)            (None, 16)                272       
                                                                 
 dense_28 (Dense)            (None, 16)                272       
                                                                 
 dense_29 (Dense)            (None, 18)                306       
                                                                 
Total params: 160,850
Trainable params: 160,850
Non-trainable params: 0
_______________________________________________

In [75]:
# We train the model by using the function fit


import numpy as np
epochs = 600
history = model.fit(padded_sequences, np.array(target_labels), epochs=epochs)

Epoch 1/600
3/3 [==============================] - 1s 7ms/step - loss: 2.8909 - accuracy: 0.0580
Epoch 2/600
3/3 [==============================] - 0s 8ms/step - loss: 2.8901 - accuracy: 0.0580
Epoch 3/600
3/3 [==============================] - 0s 8ms/step - loss: 2.8895 - accuracy: 0.0870
Epoch 4/600
3/3 [==============================] - 0s 8ms/step - loss: 2.8891 - accuracy: 0.0870
Epoch 5/600
3/3 [==============================] - 0s 8ms/step - loss: 2.8889 - accuracy: 0.0870
Epoch 6/600
3/3 [==============================] - 0s 7ms/step - loss: 2.8888 - accuracy: 0.0870
Epoch 7/600
3/3 [==============================] - 0s 11ms/step - loss: 2.8884 - accuracy: 0.0870
Epoch 8/600
3/3 [==============================] - 0s 7ms/step - loss: 2.8880 - accuracy: 0.0870
Epoch 9/600
3/3 [==============================] - 0s 8ms/step - loss: 2.8876 - accuracy: 0.0870
Epoch 10/600
3/3 [==============================] - 0s 8ms/step - loss: 2.8872 - accuracy: 0.0870
Epoch 11/600
3/3 [==========

In [76]:
# saving the model that we just trained 
model.save("chatting")

import pickle

# saving  the tokenizer that we already fitted
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# saving the label encoder that we laready fitted 
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(label_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
pip install colorama 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Implementing the chat function so that we can interact with the chtabot Hypatia!
# When the user enter a new message, Hypatia will compute the similarity between the text given and the training data 
#Based on the confidence levels of each category, the message is then connected with the highest confidence level.
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("Hypatiama.json") as file:
    data = json.load(file)


def chat():
    # loading the model 
    model = keras.models.load_model('chatting')

    # loading tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # loading label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        label_encoder = pickle.load(enc)

    # Then we define the parameter max_length
    max_length = 30
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_length))
        tag = label_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.WHITE + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.WHITE + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.BLUE + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: Hello
1/1 [==============================] - 0s 133ms/step
ChatBot: Welcome, How can I help you ?
User: Who are you 
1/1 [==============================] - 0s 21ms/step
ChatBot: I'm Hypatia, an Artificial Intelligent bot, I will try to help you with your AI questions ?
User: nice
1/1 [==============================] - 0s 22ms/step
ChatBot: The Overfitting is a situation that occurs when a model learns the training set too well, taking up random fluctuations in the training data as concepts. These impact the model’s ability to generalize and don’t apply to new data. When a model is given the training data, it shows 100 percent accuracy—technically a slight loss. But, when we use the test data, there may be an error and low efficiency. This condition is known as overfitting.
User: Thanks
1/1 [==============================] - 0s 22ms/step
ChatBot: Any time!
User: what is overfitting 
1/1 [==============================] - 0s 39ms/st